This is a notebook to keep logs on running the training procedure for the first time, with some simple preprocessing options. Have already sourced the Python 2 virtual environment set up according to instructions in the [README from the tools repo][readme] and sourced the script to set environment variables:

```
source start_script
```

Checking that this has worked by importing Theano and looking at its config:

[readme]: https://github.com/Neuroglycerin/neukrill-net-tools/blob/master/README.md

In [1]:
import theano
print(theano.config.device)

gpu0


Using gpu device 0: Tesla K40c


Looks like it's set up to use the GPU ok.

Running the code should then be as simple as pointing at the right run config json file. Using the following file:

In [2]:
cd ..

/afs/inf.ed.ac.uk/user/s08/s0805516/repos/neukrill-net-work


In [3]:
!cat run_settings/first_pylearn2_run.json

{
    "model type":"pylearn2",
    "yaml file": "default.yaml",
    "preprocessing":{"resize":[48,48]},
    "final_shape":[48,48],
    "augmentation_factor":1,
    "train_split": 0.8
}


Doing practically nothing except resizing to quite a small size. A major problem at this point is likely to be the lack of image normalization.

The YAML file specified is what controls [Pylearn2's operation][plyaml]. The file we're using has been cannabalised from the [MNIST example][mnist] bundled with Pylearn2. It's in the repo and looks like this:

[plyaml]: http://deeplearning.net/software/pylearn2/yaml_tutorial/index.html
[mnist]: http://nbviewer.ipython.org/github/lisa-lab/pylearn2/blob/master/pylearn2/scripts/tutorials/convolutional_network/convolutional_network.ipynb

In [4]:
!cat yaml_templates/default.yaml

!obj:pylearn2.train.Train {
    dataset: &train !obj:neukrill_net.dense_dataset.DensePNGDataset {
        settings_path: %(settings_path)s,
        run_settings: %(run_settings_path)s,
        training_set_mode: "train"
    },
    model: !obj:pylearn2.models.mlp.MLP {
        batch_size: 128,
        input_space: !obj:pylearn2.space.Conv2DSpace {
            shape: %(final_shape)s,
            num_channels: 1,
            axes: ['c', 0, 1, 'b'],
        },
        layers: [
                 !obj:pylearn2.models.maxout.MaxoutConvC01B {
                     layer_name: 'h0',
                     pad: 0,
                     num_channels: 48,
                     num_pieces: 2,
                     kernel_shape: [8, 8],
                     pool_shape: [4, 4],
                     pool_stride: [2, 2],
                     irange: .005,
                     max_kernel_norm: .9,
                 },
                 !obj:pylearn2.models.maxout.MaxoutConvC01B {
                     layer_name

All we need to do is point our `train.py` script (don't get this confused with either the `train` module from Pylearn2 or its own `train.py` script (unimaginative naming problems...)) at the above json:

In [ ]:
%pdb

Automatic pdb calling has been turned ON


In [ ]:
%run train.py run_settings/first_pylearn2_run.json

Input shape: (48, 48)
Detector space: (41, 41)
Output space: (20, 20)
Input shape: (20, 20)
Detector space: (19, 19)
Output space: (9, 9)
Input shape: (9, 9)
Detector space: (11, 11)
Output space: (6, 6)
Parameter and initial learning rate summary:
	h0_W: 0.0500000007451
	h0_b: 0.0500000007451
	h1_W: 0.0500000007451
	h1_b: 0.0500000007451
	h2_W: 0.0500000007451
	h2_b: 0.0500000007451
	softmax_b: 0.0500000007451
	softmax_W: 0.0500000007451
Compiling sgd_update...


/afs/inf.ed.ac.uk/user/s08/s0805516/repos/pylearn2/pylearn2/utils/iteration.py:783: UserWarning: dataset is using the old iterator interface which is deprecated and will become officially unsupported as of July 28, 2015. The dataset should implement a `get` method respecting the new interface.
  warnings.warn("dataset is using the old iterator interface which "


Compiling sgd_update done. Time elapsed: 3.973367 seconds
compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.544542 seconds
Monitored channels: 
	learning_rate
	momentum
	test_h0_kernel_norms_max
	test_h0_kernel_norms_mean
	test_h0_kernel_norms_min
	test_h0_max_x.max_u
	test_h0_max_x.mean_u
	test_h0_max_x.min_u
	test_h0_mean_x.max_u
	test_h0_mean_x.mean_u
	test_h0_mean_x.min_u
	test_h0_min_x.max_u
	test_h0_min_x.mean_u
	test_h0_min_x.min_u
	test_h0_range_x.max_u
	test_h0_range_x.mean_u
	test_h0_range_x.min_u
	test_h1_kernel_norms_max
	test_h1_kernel_norms_mean
	test_h1_kernel_norms_min
	test_h1_max_x.max_u
	test_h1_max_x.mean_u
	test_h1_max_x.min_u
	test_h1_mean_x.max_u
	test_h1_mean_x.mean_u
	test_h1_mean_x.min_u
	test_h1_min_x.max_u
	test_h1_min_x.mean_u
	test_h1_min_x.min_u
	test_h1_range_x.max_u
	test_h1_range_x.mean_u
	test_h1_range_x.min_u
	test_h2_kernel_norms_max
	test_h2_kernel_norms_mean
	test_h2_kernel_norms_min
	test_h2_max_x.max_u
	test_h2_m

Modifying Options
======

Some notes here on how options should be modified in the run_settings and to sub into YAML files.